In [1]:
#Import Libraries
from bs4 import BeautifulSoup
from html2text import html2text
import pandas as pd
import pprint as pp
import requests
import time

In [2]:
#Extract Job Title Function
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name = "div", attrs = {"class":"row"}):
        for a in div.find_all(name = "a", attrs = {"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [3]:
#Extract Job URL Function
def extract_job_url_from_result(soup): 
    urls = []
    for div in soup.find_all(name = "div", attrs = {"class":"row"}):
        for a in div.find_all(name = "a", attrs = {"data-tn-element":"jobTitle"}):
            this_url = a['href']
            to_go_url = "https://www.indeed.com/viewjob" + this_url[7:]
            urls.append(to_go_url)
    return(urls)

In [4]:
def extract_text_from_jobURL(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    
    table = soup.find('table', id = 'job-content')
    span = table.find('span', id = 'job_summary')
    div = span.find('div')
    
    return(html2text(str(div)))

In [5]:
#Extract Job Description
def extract_summary_from_result(soup): 
    summaries = []
    spans = soup.findAll("span", attrs = {"class": "summary"})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)

In [6]:
#Import the Job Title Dataset
job_titles=pd.read_csv("..\Datasets\Job_Titles.csv", sep = ",")

#Create job_titles list
job_list=list(job_titles.Job_titles)

#Replace spaces with '+'
job_list2=[]
for item in job_list:
    job_list2.append(item.replace(' ', '+'))
    
job_list=job_list2

In [7]:
#Testing the above code
URL ="https://www.indeed.com/jobs?q=data%2420%2C000&l=+New+York&start=10"
    
#conducting a request of the stated URL above:
page = requests.get(URL)

#specifying a desired format of “page” using the html parser
soup = BeautifulSoup(page.text, "html.parser")

pp.pprint(extract_job_title_from_result(soup))

pp.pprint(extract_summary_from_result(soup))

['Data Entry Operator',
 'Data Analyst',
 'Internship: Child Poverty on Violence against Children, Data & Analytic '
 'Section, DRP – NYHQ, Requisition #513831',
 'SMS/SRC Data Analyst',
 'DATA ANALYST',
 'HBO Communications Assistant',
 'Data Entry Operator',
 'Intern: Investment Data Scientist',
 'Data Analyst, Asc',
 'Data Entry']
['The role involves collecting and entering data pertaining to land and '
 'property sales deeds as well as DLD documentations....',
 'The Data Analyst will support project development and decision-making by '
 'managing, analyzing, and reporting data....',
 'The purpose of the Internship is to support data and statistics work on '
 'Child Poverty on Violence against Children including updating the global '
 'databases,...',
 'Provide technical expertise to the Manager, SRC Data Analysis regarding data '
 'models, database design development, and data mining....',
 'Responsible for setting up data pulls from multiple touch points into a '
 'database (using

In [8]:
url_list = extract_job_url_from_result(soup)

print(url_list[0])
print()
print(extract_text_from_jobURL(url_list[0]))

https://www.indeed.com/viewjob?jk=49bc5a364095bb87&fccid=52df7d9b7f5fccc3&vjs=3

This is three month temp contract role for bilingual candidates else
candidates who could read and type , would be considered. The role involves
collecting and entering data pertaining to land and property sales deeds as
well as DLD documentations. You will also maintain incident report, update all
database Keep information confidential, store completed work in designated
location.  
**Salary**  
**** 5000 per month inclusive of fixed allowances.  
**Requirements**  
****

  * High School or University Graduates with 2-4 years similar experience.
  * Good communication skills and a great attitude.
  * Essential skills: Excellent typing skills (Basic knowledge about database software is plus) MS Excel, MS Word, MS Power point (Arabic and English).


